In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from atomate2.common.flows.amorphous import MPMorphMDMaker, EquilibriumVolumeMaker

from atomate2.vasp.jobs.md import MDMaker
from atomate2.vasp.sets.core import MDSetGenerator

from pymatgen.io.vasp import Kpoints

/Users/bli/miniconda3/envs/at2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
name: str = "MP Morph vasp md"
temperature: int = 300
end_temp: int = 300
steps_convergence: int = 20
steps_quench: int = 500  # check old MPMorph code
quench_temps: dict = {
    "start_temp": 3000,
    "end_temp": 500,
    "temp_step": 500,
}
steps_production: int = 50

incar_settings = {
                    "ISPIN": 1,  # Do not consider magnetism in AIMD simulations
                    "LREAL": "Auto",  # Peform calculation in real space for AIMD due to large unit cell size
                    "LAECHG": False,  # Don't need AECCAR for AIMD
                    "EDIFFG": None,  # Does not apply to MD simulations, see: https://www.vasp.at/wiki/index.php/EDIFFG
                    "GGA": "PS",  # Just let VASP decide based on POTCAR - the default, PS yields the error below
                    "LPLANE": False,  # LPLANE is recommended to be False on Cray machines (https://www.vasp.at/wiki/index.php/LPLANE)
                    "LDAUPRINT": 0,
                }

gamma_point = user_kpoints_settings=Kpoints(
                            comment="Gamma only",
                            num_kpts=1,
                            kpts=[[0, 0, 0]],
                            kpts_weights=[1.0],
                        )
aimd_equil_maker = MDMaker(input_set_generator=MDSetGenerator(
                        ensemble="nvt",
                        start_temp=temperature,
                        end_temp=end_temp,
                        nsteps=steps_convergence,
                        time_step=2,
                        # adapted from MPMorph settings
                        user_incar_settings=incar_settings,
                        user_kpoints_settings=gamma_point,
                        ))
aimd_prod_maker = MDMaker(input_set_generator=MDSetGenerator(
                        ensemble="nvt",
                        start_temp=temperature,
                        end_temp=end_temp,
                        nsteps=steps_production,
                        time_step=2,
                        # adapted from MPMorph settings
                        user_incar_settings=incar_settings,
                        user_kpoints_settings=gamma_point,
                        ))


equil_vol = EquilibriumVolumeMaker(eos_relax_maker=aimd_equil_maker)

In [4]:
from mp_api.client import MPRester
with MPRester(api_key="jATnFCFc6afcHi9vTozmteJAYjdFIFbS") as mpr:
    structure = mpr.get_structure_by_material_id('mp-149', conventional_unit_cell=True)

No module named 'phonopy'
No module named 'phonopy'


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 2816.86it/s]


In [10]:
flow = equil_vol.make(structure)

new print test
None
new print test
OutputReference(ca41fd7c-f6f4-4878-8e01-d3c8a7e698fb)


TypeError: unsupported operand type(s) for /: 'OutputReference' and 'float'